Combine 2 datasets and create the fact and dimensions

In [1223]:
import pandas as pd

# Load the provided CSV files
file1 = "cleaned_Computed insight - Success of active sellers.csv"
file2 = "cleaned_summer-products-with-rating-and-performance_2020-08.csv"

data1 = pd.read_csv(file1)
data2 = pd.read_csv(file2)

print(data1.columns)
print(data2.columns)

# Combine the datasets using the corrected column name
combined_data = pd.merge(data1, data2, how="inner", on="merchant_id")

# Save the combined dataset
combined_file = "combined_dataset.csv"
combined_data.to_csv(combined_file, index=False)

Index(['index', 'merchant_id', 'listed_products', 'total_units_sold',
       'mean_units_sold_per_product', 'rating_per_merchant_products',
       'mean_product_prices', 'mean_retail_prices', 'average_discount',
       'revenues'],
      dtype='object')
Index(['index', 'title', 'title_orig', 'price', 'retail_price',
       'currency_buyer', 'units_sold', 'rating', 'additional_information',
       'product_color', 'product_variation_inventory', 'shipping_option_name',
       'shipping_option_price', 'countries_shipped_to',
       'inventory_total_start', 'origin_country', 'merchant_title',
       'merchant_name', 'merchant_info_subtitle', 'merchant_id',
       'merchant_has_profile_picture', 'product_url', 'product_picture',
       'product_id', 'ratings_over_three', 'ratings_below_three',
       'inventory_total_end', 'origin_country_id', 'profile_id',
       'shipping_id'],
      dtype='object')


In [1224]:
# Split the data into the required tables based on the schema

# Fact Table: Inventory_Fact
inventory_fact_columns = [
    'product_id', 'merchant_id', 'shipping_id', 'inventory_total_start',
    'inventory_total_end', 'product_variation_inventory',
    'units_sold'
]
inventory_fact = combined_data[inventory_fact_columns]

# Dim_Product
dim_product_columns = [
    'product_id', 'price', 'retail_price', 'rating', 'title', 'title_orig',
    'product_color', 'countries_shipped_to', 'additional_information', 'origin_country_id',
    'ratings_over_three', 'ratings_below_three'
]
dim_product = combined_data[dim_product_columns]

# Dim_Product_Origin_Country
dim_product_origin_country_columns = ['origin_country_id', 'origin_country']
dim_product_origin_country = combined_data[dim_product_origin_country_columns]

# Dim_Merchant
dim_merchant_columns = [
    'merchant_id', 'merchant_title', 'merchant_name', 'revenues', 'rating_per_merchant_products',
    'listed_products', 'mean_product_prices', 'mean_retail_prices', 'mean_units_sold_per_product',
    'average_discount', 'profile_id'
]
dim_merchant = combined_data[dim_merchant_columns]

# Dim_Profile
dim_profile_columns = ['profile_id', 'merchant_has_profile_picture', 'merchant_info_subtitle']
dim_profile = combined_data[dim_profile_columns]

# Dim_Shipping
dim_shipping_columns = ['shipping_id', 'shipping_option_name', 'shipping_option_price']
dim_shipping = combined_data[dim_shipping_columns]

# Save all tables as CSV files
output_dir = r"C:\Users\Mokhmed\Downloads"

inventory_fact.to_csv(r"C:\Users\Mokhmed\Downloads\Inventory_Fact.csv", index=False)
dim_product.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Product.csv", index=False)
dim_product_origin_country.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Product_Origin_Country.csv", index=False)
dim_merchant.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Merchant.csv", index=False)
dim_profile.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Profile.csv", index=False)
dim_shipping.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Shipping.csv", index=False)

print("Combined dataset and dimension/fact CSV files have been generated!")

Combined dataset and dimension/fact CSV files have been generated!


Handling duplicates in dim profile

In [1225]:
# Check for entire duplicates
dim_profile = pd.read_csv(r"C:\Users\Mokhmed\Downloads\Dim_Profile.csv")
duplicates = dim_profile[dim_profile.duplicated()]
duplicates

,profile_id,merchant_has_profile_picture,merchant_info_subtitle
1,10,1,"90 % Positive Feedback (320,031 ratings)"
3,218,0,"84 % Positive Feedback (139,223 ratings)"
4,218,0,"84 % Positive Feedback (139,223 ratings)"
5,218,0,"84 % Positive Feedback (139,223 ratings)"
6,218,0,"84 % Positive Feedback (139,223 ratings)"
...,...,...,...
1533,912,0,79 % Positive Feedback (42 ratings)
1535,856,0,79 % Positive Feedback (291 ratings)
1537,917,0,85 % Positive Feedback (13 ratings)
1539,613,0,80 % Positive Feedback (129 ratings)


In [1226]:
dim_profile = dim_profile.drop_duplicates()
entire_duplicates = dim_profile[dim_profile.duplicated()]
print(entire_duplicates)

Empty DataFrame
Columns: [profile_id, merchant_has_profile_picture, merchant_info_subtitle]
Index: []


In [1227]:
#Check for partial duplicates
partial_duplicates = (
    dim_profile.groupby('profile_id')
    .filter(lambda group: 'No feedback available' in group['merchant_info_subtitle'].values 
                          and len(group['merchant_info_subtitle'].unique()) > 1)
)

# Display the partial duplicates
print("Partial duplicates:")
partial_duplicates

Partial duplicates:


,profile_id,merchant_has_profile_picture,merchant_info_subtitle
12,516,0,No feedback available
13,516,0,"89 % Positive Feedback (55,499 ratings)"
19,404,0,"86 % Positive Feedback (151,249 ratings)"
21,404,0,No feedback available
23,5,0,No feedback available
...,...,...,...
1192,890,0,No feedback available
1211,632,0,No feedback available
1212,632,0,"83 % Positive Feedback (9,703 ratings)"
1297,182,0,No feedback available


In [1228]:
#Handling cases where a profile has both no feedback available and an actual percentage
def filter_duplicates(group):
    # If 'No feedback available' exists alongside other values, drop it
    if 'No feedback available' in group['merchant_info_subtitle'].values and len(group) > 1:
        return group[group['merchant_info_subtitle'] != 'No feedback available']
    # Otherwise, return the group as is
    return group

# Group by profile_id and apply the filter logic
dim_profile = dim_profile.groupby('profile_id', group_keys=False).apply(filter_duplicates)

# Checks for duplicates again
duplicates = dim_profile[dim_profile.duplicated(subset='profile_id', keep=False)]
duplicates

C:\Users\Mokhmed\AppData\Local\Temp\ipykernel_13440\3890542993.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dim_profile = dim_profile.groupby('profile_id', group_keys=False).apply(filter_duplicates)


,profile_id,merchant_has_profile_picture,merchant_info_subtitle
47,18,0,"88 % Positive Feedback (151,914 ratings)"
50,18,0,"88% Positive Feedback (151,914 ratings)"
57,107,0,"83 % Positive Feedback (32,168 ratings)"
59,107,0,"Positive Feedback 83% (32,168 ratings)"
1205,121,0,"83% Positive Feedback (3,237 ratings)"
1206,121,0,"83 % Positive Feedback (3,237 ratings)"
240,144,0,"83 % Positive Feedback (17,752 ratings)"
242,144,0,"83% Positive Feedback (17,752 ratings)"
414,168,0,"83 % Positive Feedback (5,083 ratings)"
416,168,0,"83% Positive Feedback (5,083 ratings)"


In [1229]:
#Handling the duplicates that are left
dim_profile = dim_profile.drop_duplicates(subset='profile_id')
duplicates = dim_profile[dim_profile.duplicated(subset='profile_id', keep=False)]
duplicates

,profile_id,merchant_has_profile_picture,merchant_info_subtitle


In [1230]:
# Check if there are different values for "merchant_has_profile_picture" within the same "profile_id"

inconsistent_profiles = dim_profile.groupby('profile_id')['merchant_has_profile_picture'].nunique()
inconsistent_profiles = inconsistent_profiles[inconsistent_profiles > 1]

if inconsistent_profiles.empty:
    print("All profiles have consistent values for 'merchant_has_profile_picture'.")
else:
    print("Inconsistent profiles detected:", inconsistent_profiles)


All profiles have consistent values for 'merchant_has_profile_picture'.


In [1231]:
dim_profile_cleaned = dim_profile[['profile_id', 'merchant_has_profile_picture','merchant_info_subtitle']]
dim_profile_cleaned.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Profile_Cleaned.csv", index=False)

Handling duplicates in dim product

In [1232]:
# Check for duplicates in product_id
dim_product = pd.read_csv(r"C:\Users\Mokhmed\Downloads\Dim_Product.csv")
duplicates = dim_product[dim_product.duplicated(subset='product_id', keep=False)]
duplicates

,product_id,price,retail_price,rating,title,title_orig,product_color,countries_shipped_to,additional_information,origin_country_id,ratings_over_three,ratings_below_three
16,5ca084765e18c31890903f2a,3.92,4,3.76,2019 Mode d'été pour dames - Pantalon à lacets...,2019 Summer Women‘s’ Fashion Lace Up Tie Pant...,black,36,"Summer,Plus Size,high waist,pants,Waist,Short ...",2,63.384474,20.998397
17,5ca084765e18c31890903f2a,3.92,4,3.76,2019 Mode d'été pour dames - Pantalon à lacets...,2019 Summer Women‘s’ Fashion Lace Up Tie Pant...,black,36,"Summer,Plus Size,high waist,pants,Waist,Short ...",2,63.384474,20.998397
40,5e16cb87e6dd7c03be24b28a,2.00,2,4.19,Femmes d'été Mode Débardeurs Sexy Sans Manches...,Women Summer Fashion Tank Tops Sexy Sleeveless...,green,35,"blouse,Summer,Plus Size,Tops & Blouses,Lace,Ta...",2,79.012346,13.580247
41,5e16cb87e6dd7c03be24b28a,2.00,2,4.19,Femmes d'été Mode Débardeurs Sexy Sans Manches...,Women Summer Fashion Tank Tops Sexy Sleeveless...,green,35,"blouse,Summer,Plus Size,Tops & Blouses,Lace,Ta...",2,79.012346,13.580247
42,5e16cb87e6dd7c03be24b28a,2.00,2,4.19,Femmes d'été Mode Débardeurs Sexy Sans Manches...,Women Summer Fashion Tank Tops Sexy Sleeveless...,green,35,"blouse,Summer,Plus Size,Tops & Blouses,Lace,Ta...",2,79.012346,13.580247
...,...,...,...,...,...,...,...,...,...,...,...,...
1537,5edf2eac07570835cf35ee02,2.67,67,5.00,Papillon Imprimer Coton Masque Visage Sport En...,Butterfly Print Cotton Face Mask Outdoor Sport...,purple,27,"butterfly,Summer,Outdoor,isolation,Masks,Sport...",2,66.666667,18.314118
1538,5ed1c059630f990d1d44592c,5.00,5,5.00,Jupe de tennis plissée extensible Active Skort...,Women's Active Skort Athletic Stretchy Pleated...,white,8,"ladies skort,running skirt,summer skirt,Golf,s...",2,66.666667,18.314118
1539,5ed1c059630f990d1d44592c,5.00,5,5.00,Jupe de tennis plissée extensible Active Skort...,Women's Active Skort Athletic Stretchy Pleated...,white,8,"ladies skort,running skirt,summer skirt,Golf,s...",2,66.666667,18.314118
1540,5ea69521f5e1e65a3c70faa8,7.00,68,1.50,Mode féminine été bretelles spaghetti imprimé ...,Women's Summer Fashion Spaghetti Strap Floral ...,white,43,"Mini,spaghettistrapdresse,Floral print,Summer,...",2,0.000000,100.000000


In [1233]:
# Check if the duplicates are partial (by grouping by 'product_id' and looking at unique values in the other columns)

# Getting the count of unique values in other columns
duplicate_groups = dim_product.groupby('product_id').nunique()

# Filtering for rows where any column has more than 1 unique value
partial_duplicates = duplicate_groups[duplicate_groups > 1]

# Calculating how many product_ids have partial duplicates
partial_duplicates_count = partial_duplicates.count().sum()

print(f"There are {partial_duplicates_count} partial duplicates in the dataset.")

There are 0 partial duplicates in the dataset.


In [1234]:
dim_product = dim_product.drop_duplicates(subset=['product_id'])

In [1235]:
duplicates = dim_product[dim_product.duplicated(subset='product_id', keep=False)]
duplicates

,product_id,price,retail_price,rating,title,title_orig,product_color,countries_shipped_to,additional_information,origin_country_id,ratings_over_three,ratings_below_three


In [1236]:
dim_product_cleaned = dim_product[['product_id', 'price','retail_price','rating','title','title_orig','product_color','countries_shipped_to','additional_information','origin_country_id']].drop_duplicates()
dim_product_cleaned.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Product_Cleaned.csv", index=False)

duplicates = dim_product_cleaned[dim_product_cleaned.duplicated(subset='product_id', keep=False)]
duplicates

,product_id,price,retail_price,rating,title,title_orig,product_color,countries_shipped_to,additional_information,origin_country_id


In [1237]:
duplicates = dim_product_cleaned[dim_product_cleaned.duplicated()]
duplicates

,product_id,price,retail_price,rating,title,title_orig,product_color,countries_shipped_to,additional_information,origin_country_id


Handling duplicates in dim merchant

In [1238]:
# Check for duplicates in merchant_id
dim_merchant = pd.read_csv(r"C:\Users\Mokhmed\Downloads\Dim_Merchant.csv")
duplicates = dim_merchant[dim_merchant.duplicated(subset='merchant_id', keep=False)]
duplicates

,merchant_id,merchant_title,merchant_name,revenues,rating_per_merchant_products,listed_products,mean_product_prices,mean_retail_prices,mean_units_sold_per_product,average_discount,profile_id
0,5357bcf2bb72c5504882e889,ApparelHeaven,primesalecolimited,1080000.0,4.219,2,9.00,20.0,60000.0,54.0,10
1,5357bcf2bb72c5504882e889,ApparelHeaven,primesalecolimited,1080000.0,4.219,2,9.00,20.0,60000.0,54.0,10
2,5708773c3c02161b3f8c7900,leiston,leiston,831096.0,3.934,5,7.76,34.2,21420.0,61.0,218
3,5708773c3c02161b3f8c7900,leiston,leiston,831096.0,3.934,5,7.76,34.2,21420.0,61.0,218
4,5708773c3c02161b3f8c7900,leiston,leiston,831096.0,3.934,5,7.76,34.2,21420.0,61.0,218
...,...,...,...,...,...,...,...,...,...,...,...
1537,5e606b6e29e7867135073fbe,hehaiyan,hehaiyan,53.4,3.692,2,2.67,67.0,10.0,97.0,917
1538,5a3231f2e62bbc45b0bda1dd,renpingping,renpingping,100.0,3.837,2,5.00,5.0,10.0,0.0,613
1539,5a3231f2e62bbc45b0bda1dd,renpingping,renpingping,100.0,3.837,2,5.00,5.0,10.0,0.0,613
1540,5d48eef4b5144c50c2ac0af3,wenj498,wenj498,140.0,3.859,2,7.00,68.0,10.0,90.0,818


In [1239]:
# Check if the duplicates are partial (by grouping by 'merchant_id' and looking at unique values in the other columns)

# Getting the count of unique values in other columns
duplicate_groups = dim_merchant.groupby('merchant_id').nunique()

# Filtering for rows where any column has more than 1 unique value
partial_duplicates = duplicate_groups[duplicate_groups > 1]

# Calculating how many merchant_ids have partial duplicates
partial_duplicates_count = partial_duplicates.count().sum()

print(f"There are {partial_duplicates_count} partial duplicates in the dataset.")


There are 0 partial duplicates in the dataset.


In [1240]:
dim_merchant = dim_merchant.drop_duplicates(subset=['merchant_id'])
duplicates = dim_merchant[dim_merchant.duplicated(subset='merchant_id', keep=False)]
duplicates

,merchant_id,merchant_title,merchant_name,revenues,rating_per_merchant_products,listed_products,mean_product_prices,mean_retail_prices,mean_units_sold_per_product,average_discount,profile_id


In [1241]:
dim_merchant_cleaned = dim_merchant[['merchant_id', 'merchant_title','merchant_name','revenues','rating_per_merchant_products','listed_products','mean_product_prices','mean_retail_prices','mean_units_sold_per_product','average_discount','profile_id']].drop_duplicates()
dim_merchant_cleaned.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Merchant_Cleaned.csv", index=False)

duplicates = dim_merchant_cleaned[dim_merchant_cleaned.duplicated(subset='merchant_id', keep=False)]
duplicates

,merchant_id,merchant_title,merchant_name,revenues,rating_per_merchant_products,listed_products,mean_product_prices,mean_retail_prices,mean_units_sold_per_product,average_discount,profile_id


In [1242]:
duplicates = dim_merchant_cleaned[dim_merchant_cleaned.duplicated()]
duplicates

,merchant_id,merchant_title,merchant_name,revenues,rating_per_merchant_products,listed_products,mean_product_prices,mean_retail_prices,mean_units_sold_per_product,average_discount,profile_id


Handling duplicates in dim shipping

In [1243]:
dim_shipping = pd.read_csv(r"C:\Users\Mokhmed\Downloads\Dim_Shipping.csv")
duplicates = dim_shipping[dim_shipping.duplicated(subset='shipping_id', keep=False)]
duplicates

,shipping_id,shipping_option_name,shipping_option_price
0,2,Standard shipping,2
1,2,Standard shipping,3
2,2,Standard shipping,1
3,2,Standard shipping,3
4,2,Standard shipping,4
...,...,...,...
1568,2,Standard shipping,3
1569,2,Standard shipping,5
1570,2,Standard shipping,1
1571,2,Standard shipping,4


In [1244]:
dim_shipping = pd.DataFrame(dim_shipping)

# Group by shipping_id and shipping_option_name to calculate the mean price
cleaned_dim_shipping = dim_shipping.groupby(['shipping_id', 'shipping_option_name'], as_index=False).agg({
    'shipping_option_price': 'mean'
})

# Print the cleaned dataset
print(cleaned_dim_shipping)

cleaned_dim_shipping.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Shipping_Cleaned.csv", index=False)

   shipping_id shipping_option_name  shipping_option_price
0            1     Express shipping               8.000000
1            2    Standard shipping               2.330784


Handling duplicates in inventory fact

In [1245]:
inventory_fact

,product_id,merchant_id,shipping_id,inventory_total_start,inventory_total_end,product_variation_inventory,units_sold
0,567cb0925ff12c4829f07dfb,5357bcf2bb72c5504882e889,2,101000.0,1000.0,50,100000
1,572981a997dc5f5d3c5f0651,5357bcf2bb72c5504882e889,2,21000.0,1000.0,50,20000
2,5b07a7c6e997b33c6f9947da,5708773c3c02161b3f8c7900,2,101000.0,1000.0,50,100000
3,5e9692898a10221b310986c9,5708773c3c02161b3f8c7900,2,1100.0,1000.0,17,100
4,5cece16810467a5f254abaa9,5708773c3c02161b3f8c7900,2,2000.0,1000.0,50,1000
...,...,...,...,...,...,...,...
1568,5ed8983fb83b7828822ad26a,567bad8582c35f28103e5813,2,1002.0,1000.0,50,2
1569,5f11698e9a44cd5ecd1909db,5b24edf0d1c360301099e9c6,2,1002.0,1000.0,10,2
1570,5ed3ae8e7bf8821d5cf4bc64,5d417e7070327a2743021677,2,1001.0,1000.0,1,1
1571,5eccb455d4eb5d46b00be0ed,5add589edb5f1f32114d07df,2,1001.0,1000.0,5,1


In [1246]:
inventory_fact = pd.read_csv(r"C:\Users\Mokhmed\Downloads\Inventory_Fact.csv")
duplicates = inventory_fact[inventory_fact.duplicated(subset='product_id', keep=False)]
duplicates

,product_id,merchant_id,shipping_id,inventory_total_start,inventory_total_end,product_variation_inventory,units_sold
16,5ca084765e18c31890903f2a,5926c5ace8ff5525241b368d,2,21000.0,1000.0,7,20000
17,5ca084765e18c31890903f2a,5926c5ace8ff5525241b368d,2,21000.0,1000.0,7,20000
40,5e16cb87e6dd7c03be24b28a,58ad449708de0c6dc59d9e06,2,2000.0,1000.0,1,1000
41,5e16cb87e6dd7c03be24b28a,58ad449708de0c6dc59d9e06,2,2000.0,1000.0,1,1000
42,5e16cb87e6dd7c03be24b28a,58ad449708de0c6dc59d9e06,2,2000.0,1000.0,1,1000
...,...,...,...,...,...,...,...
1537,5edf2eac07570835cf35ee02,5e606b6e29e7867135073fbe,2,1010.0,1000.0,5,10
1538,5ed1c059630f990d1d44592c,5a3231f2e62bbc45b0bda1dd,2,1010.0,1000.0,50,10
1539,5ed1c059630f990d1d44592c,5a3231f2e62bbc45b0bda1dd,2,1010.0,1000.0,50,10
1540,5ea69521f5e1e65a3c70faa8,5d48eef4b5144c50c2ac0af3,2,1010.0,1000.0,50,10


In [1247]:
# Check for partial duplicates (by grouping by 'product_id' and looking at unique values in the other columns)

# Geting the count of unique values in other columns
duplicate_groups = inventory_fact.groupby('product_id').nunique()

# Filtering for rows where any column has more than 1 unique value
partial_duplicates = duplicate_groups[duplicate_groups > 1]

# Calculating how many product_ids have partial duplicates
partial_duplicates_count = partial_duplicates.count().sum()

print(f"There are {partial_duplicates_count} partial duplicates in the dataset.")

There are 0 partial duplicates in the dataset.


In [1248]:
inventory_fact_cleaned = inventory_fact[['product_id','merchant_id','shipping_id','inventory_total_start',
'inventory_total_end','product_variation_inventory','units_sold']].drop_duplicates(subset=['product_id'])
inventory_fact_cleaned.to_csv(r"C:\Users\Mokhmed\Downloads\Inventory_Fact_Cleaned.csv", index=False)

duplicates = inventory_fact_cleaned[inventory_fact_cleaned.duplicated(subset='product_id', keep=False)]
duplicates

,product_id,merchant_id,shipping_id,inventory_total_start,inventory_total_end,product_variation_inventory,units_sold


In [1249]:
duplicates = inventory_fact_cleaned[inventory_fact_cleaned.duplicated()]
duplicates

,product_id,merchant_id,shipping_id,inventory_total_start,inventory_total_end,product_variation_inventory,units_sold


Verifying that each Merchant can have multiple products

In [1250]:
duplicates2 = inventory_fact[inventory_fact.duplicated(subset='merchant_id', keep=False)]
duplicates2

,product_id,merchant_id,shipping_id,inventory_total_start,inventory_total_end,product_variation_inventory,units_sold
0,567cb0925ff12c4829f07dfb,5357bcf2bb72c5504882e889,2,101000.0,1000.0,50,100000
1,572981a997dc5f5d3c5f0651,5357bcf2bb72c5504882e889,2,21000.0,1000.0,50,20000
2,5b07a7c6e997b33c6f9947da,5708773c3c02161b3f8c7900,2,101000.0,1000.0,50,100000
3,5e9692898a10221b310986c9,5708773c3c02161b3f8c7900,2,1100.0,1000.0,17,100
4,5cece16810467a5f254abaa9,5708773c3c02161b3f8c7900,2,2000.0,1000.0,50,1000
...,...,...,...,...,...,...,...
1537,5edf2eac07570835cf35ee02,5e606b6e29e7867135073fbe,2,1010.0,1000.0,5,10
1538,5ed1c059630f990d1d44592c,5a3231f2e62bbc45b0bda1dd,2,1010.0,1000.0,50,10
1539,5ed1c059630f990d1d44592c,5a3231f2e62bbc45b0bda1dd,2,1010.0,1000.0,50,10
1540,5ea69521f5e1e65a3c70faa8,5d48eef4b5144c50c2ac0af3,2,1010.0,1000.0,50,10


Handling duplicates in dim origin country

In [1251]:
dim_origin_country = pd.read_csv(r"C:\Users\Mokhmed\Downloads\Dim_Product_Origin_Country.csv")
duplicates = dim_origin_country[dim_origin_country.duplicated(subset='origin_country_id', keep=False)]
duplicates

,origin_country_id,origin_country
0,2,CN
1,2,CN
2,2,CN
3,2,CN
4,2,CN
...,...,...
1568,2,CN
1569,2,CN
1570,2,CN
1571,2,CN


In [1252]:
# Check for partial duplicates (by grouping by 'origin_country_id' and looking at unique values in the other columns)

# Geting the count of unique values in other columns
duplicate_groups = dim_origin_country.groupby('origin_country_id').nunique()

# Filtering for rows where any column has more than 1 unique value
partial_duplicates = duplicate_groups[duplicate_groups > 1]

# Calculating how many product_ids have partial duplicates
partial_duplicates_count = partial_duplicates.count().sum()

print(f"There are {partial_duplicates_count} partial duplicates in the dataset.")

There are 0 partial duplicates in the dataset.


In [1253]:
dim_origin_country_cleaned = dim_origin_country[['origin_country_id','origin_country']].drop_duplicates(subset=['origin_country_id'])
dim_origin_country_cleaned.to_csv(r"C:\Users\Mokhmed\Downloads\Dim_Product_Origin_Country_Cleaned.csv", index=False)

duplicates = dim_origin_country_cleaned[dim_origin_country_cleaned.duplicated(subset='origin_country_id', keep=False)]
duplicates

,origin_country_id,origin_country


In [1254]:
duplicates = dim_origin_country_cleaned[dim_origin_country_cleaned.duplicated()]
duplicates

,origin_country_id,origin_country
